In [16]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from models.cnn import CNNImageForgeryDetector
from models.svm import SVMImageForgeryDetector
from models.fine_tuning import TransferLearningImageForgeryDetector

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 2.0 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/12.9 MB 2.3 MB/s eta 0:00:06
   ----- ---------------------------------- 1.8/12.9 MB 2.3 MB/s eta 0:00:05
   ----- ---------------------------------- 1.8/12.9 MB 2.3 MB/s eta 0:00:05
   ----- ---------------------------------- 1.8/12.9 MB 2.3 MB/s eta 0:00:05
   ---------- ----------------------------- 3.4/12.9 MB 2.6 MB/s eta 0:00:04
   ---------- ----------------------------- 3.4/12.9 MB 2.6 MB/s eta 0:00:04
   ------------- -------------------------- 4.2/12.9 MB 2.3 MB/s eta 0:00:04
   ------------- -------------------------- 4.2/12.9 MB 2.3 MB/s eta 0:00:04
   ------------- -------------------------- 4.2/12.9 MB 2.3 MB/s eta 0:00:04
   -----------------

In [18]:
base_models = [
    ('cnn', CNNImageForgeryDetector(img_size=(128, 128), compression_quality=80, learning_rate=0.0001)),
    ('transfer_learning', TransferLearningImageForgeryDetector(img_size=(128, 128), learning_rate=0.0001)),
    ('svm', SVMImageForgeryDetector(use_edges=False, use_noise=False, use_texture=False, img_size=(128, 128)))
]

9406464/9406464 [==============================] - 4s 0us/step


In [19]:
from sklearn.linear_model import LogisticRegression
meta_model = LogisticRegression()


In [20]:
from sklearn.ensemble import StackingClassifier

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)


In [23]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [30]:
import os
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

In [31]:
real_images_dir = 'C:/MachineLearning/ml/Edited-Photo-Detection/data/CASIA2/Au'
edited_images_dir = 'C:/MachineLearning/ml/Edited-Photo-Detection/data/CASIA2/Tp'

In [34]:
IMG_SIZE = (128, 128)
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert('RGB')  # Cargar imagen en modo RGB
            img = img.resize(IMG_SIZE)  # Redimensionar la imagen
            img_array = np.array(img) / 255.0  # Convertir a array y normalizar entre 0 y 1
            images.append(img_array)
            labels.append(label)
    return images, labels

# Cargar imágenes reales y editadas
real_images, real_labels = load_images_from_folder(real_images_dir, label=0)  # 0 para reales
edited_images, edited_labels = load_images_from_folder(edited_images_dir, label=1) 

In [35]:
images = real_images + edited_images
labels = real_labels + edited_labels

In [36]:
# Convertir a numpy arrays
images = np.array(images)
labels = np.array(labels)

# Dividir el dataset en train, validation y test
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42, stratify=labels)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Total imágenes de entrenamiento: {len(X_train)}")
print(f"Total imágenes de validación: {len(X_val)}")
print(f"Total imágenes de prueba: {len(X_test)}")

Total imágenes de entrenamiento: 8792
Total imágenes de validación: 1884
Total imágenes de prueba: 1884


In [37]:
import numpy as np

# Contar cuántas imágenes reales y editadas hay en cada conjunto
def count_labels(y_data):
    unique, counts = np.unique(y_data, return_counts=True)
    return dict(zip(unique, counts))

# Contar en cada conjunto (0 = reales, 1 = editadas)
train_counts = count_labels(y_train)
val_counts = count_labels(y_val)
test_counts = count_labels(y_test)

# Mostrar los resultados
print(f"Conjunto de entrenamiento: {train_counts}")
print(f"Conjunto de validación: {val_counts}")
print(f"Conjunto de prueba: {test_counts}")


Conjunto de entrenamiento: {0: 5206, 1: 3586}
Conjunto de validación: {0: 1115, 1: 769}
Conjunto de prueba: {0: 1116, 1: 768}


In [38]:
stacking_model.fit(X_train, y_train)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [22]:
y_pred = stacking_model.predict(X_test)
print_model_performance_metrics(y_test, y_pred)

NameError: name 'X_test' is not defined